## Анализ поведения пользователей мобильного приложения
##  Описание данных:
Датасет содержит данные о событиях, совершенных в мобильном приложении "Ненужные вещи". В нем пользователи продают свои ненужные вещи, размещая их на доске объявлений. В датасете содержатся данные пользователей, впервые совершивших действия в приложении после 7 октября 2019 года.

Колонки в /datasets/mobile_sources.csv:
userId — идентификатор пользователя,
source — источник, с которого пользователь установил приложение.

Колонки в /datasets/mobile_dataset.csv:
event.time — время совершения,
user.id — идентификатор пользователя, event.name — действие пользователя.   
  
Виды действий:
advert_open — открыл карточки объявления,
photos_show — просмотрел фотографий в объявлении,
tips_show — увидел рекомендованные объявления,
tips_click — кликнул по рекомендованному объявлению,
contacts_show и show_contacts — посмотрел номер телефона,
contacts_call — позвонил по номеру из объявления,
map — открыл карту объявлений,
search_1 — search_7 — разные действия, связанные с поиском по сайту,
favorites_add — добавил объявление в избранное.

## Цели исследования
1. Управление вовлеченностью клиентов (адаптация приложения по целевой и смежной
аудитории) будет актуально только на основе данных о поведении пользователей.
2. Получить на основе поведения пользователей гипотезы о том как можно было бы улучшить
 приложение с точки зрения пользовательского опыта.

## Задачи:
1. Проанализировать связь целевого события — просмотра контактов — и других действий пользователей.
2. Оценить, какие действия чаще совершают те пользователи, которые просматривают контакты.
## Основные вопросы исследования
1. Проанализируйте связь целевого события —
просмотра контактов — и других действий
пользователей:
1.1. В разрезе сессий отобрать сценарии\паттерны, которые
приводят к просмотру контактов
1.2. Построить воронки по основным сценариям в разрезе
уникальных пользователей
2. Оцените, какие действия чаще совершают те
пользователи, которые просматривают контакты:
2.1. Рассчитать относительную частоту событий в разрезе двух
групп пользователей:
- группа пользователей, которые смотрели
контакты contacts_show
-  группа пользователей, которые не смотрели
контакты contacts_show

## Этапы исследования  ##
  
1. Изучение данных  
2. Предобработка  
- Переименование столбцов  
- Изменение типов данных   
- Проверка на явные и неявные дубликаты  
- Добавление столбцов  
- Проверка на аномальные значения  

3. Исследовательский анализ   
- Анализ активности пользователей    
- Определение а анализ сессий  
4. Какие действие совершают пользователи которые просматривают контакты   
5. Проверка гипотез  
6. Выводы  


##  Изучение данных

In [1]:

import numpy as np
import pandas as pd
import seaborn as sns
from pandas.plotting import register_matplotlib_converters
from plotly import graph_objects as go
from datetime import datetime
from datetime import timedelta

import matplotlib.pyplot as plt
register_matplotlib_converters()
import warnings
warnings.filterwarnings('ignore')

In [2]:
import tqdm
import requests
from tqdm import tqdm
tqdm.pandas()

In [3]:
mobile_data = pd.read_csv('mobile_dataset.csv') 
mobile_sources = pd.read_csv('mobile_sources.csv') 


FileNotFoundError: [Errno 2] No such file or directory: 'mobile_dataset.csv'

In [ ]:
mobile_data.head(10)
mobile_data.info()

В таблице mobile_data содержится информация о  74197 записях посещения приложения пользователями

In [ ]:
mobile_sources.head(10)
mobile_sources.info()

In [ ]:
mobile_sources.describe()

В таблице 4293 записи. В колонке source - источники привлечения три уникальных записи. 

In [ ]:
print('Пользователи привлекаются из трех типов источников: ',', '\
      .join(mobile_sources['source'].unique().tolist()))

In [ ]:
print('Типы событий: ')
mobile_data['event.name'].unique()


In [ ]:
mobile_sources.info()

In [ ]:
print('Общее количество событий в логе:', mobile_data['event.name'].count())
print('Количество пользователей в логе:', len(mobile_data['user.id'].unique()))

print('Дата начала эксперимента:', mobile_data['event.time'].min())
print('Дата конца эксперимента:', mobile_data['event.time'].max())


В списке событий есть дубликаты, далее их обработаем.

**Выводы по результатам обзора данных.** 
- В таблице mobile_data содержится информация о  74197 записях посещения приложения пользователями. В таблице 5 стоблцов:event.time, event.name и user.id. 
В event.name представлены события, которые совершают пользователи на сайта. Всего 9 уникальных события, но в столбце есть дубликаты - далее их обработаем.

- В таблице mobile_sources - информацию о 4293 записях в 2-х столбцах user_id и source. В source - инфомация о трех источниках привлечения - yandex, google, other
В таблицах нет пропущенных значений

## Предобработка данных ##

**Этапы предобработки**  
1. Переименование столбцов  
2. Изменения типа данных  
3. Проверка на скрытые и явные дубликаты  
4. Добавление столбцов  
5. Поиск аномалий   

### Переименование столбцов

In [ ]:
# переименование столбцов
mobile_sources = mobile_sources.rename(columns={'userId': 'user_id'})
mobile_data = mobile_data.rename(columns={'event.time': 'event_time','event.name': 'event_name',\
                                          'user.id': 'user_id'})

### Изменение типа данных

In [ ]:
# меняем тип данных у столбца event_time в таблице 
mobile_data['event_time'] = pd.to_datetime(mobile_data['event_time']).round('ms')


### Проверка на дубликаты

In [ ]:
# проверка на скрытые дубликаты
print(mobile_data['event_name'].unique())

В выводе есть дубликаты действия - search и show_contacts/contacts_show. Дубликаты убираем. Оставляем show_contacts и search соответственно.

In [ ]:

mobile_data['event_name']=mobile_data['event_name'].str.replace('search_\d+', 'search',regex = True)
mobile_data['event_name']=mobile_data['event_name'].str\
                                            .replace('contacts_show', 'show_contacts',regex = True)

In [ ]:
print(mobile_data['event_name'].unique())

Получаем 9 действий, которые делает пользователь. Целевое действие - просмотр контактов/ show_contacts

In [ ]:
# проверяем на скрытые дубликаты 
print('Дубликаты в mobile_data: ', mobile_data.duplicated().sum())
print('Дубликаты в mobile_sources:', mobile_sources.duplicated().sum())


In [ ]:
# удаляем дубликаты из таблицы mobile_data
mobile_data.drop_duplicates(inplace=True)
mobile_data.reset_index(inplace=True, drop=True)
print('Дубликаты в mobile_data: ', mobile_data.duplicated().sum())

### Добавление столбцов

In [ ]:
# добавляем столбцы 
mobile_data['event_datetime'] = pd.to_datetime(mobile_data['event_time'], unit='ms')
mobile_data['event_date'] = mobile_data['event_time'].dt.date
mobile_data['day_of_week'] = mobile_data['event_time'].dt.day_of_week

In [ ]:
mobile_data.head(10)

### Проверка на аномалии числа событий на пользователя

In [ ]:
# считаем количество событий на пользователя
user_event_table = mobile_data.groupby('user_id')['event_name'].size()
user_event_table = pd.DataFrame(user_event_table)
user_event_table.columns = ['count_event']
user_event_table.sort_values('count_event', inplace=True)

statistics = user_event_table['count_event'].describe()
median = statistics['50%']
min = statistics.min()
print('Медианное число событий на пользователя:', median)

In [ ]:
plt.figure(figsize=(14, 4))
plt.boxplot(user_event_table['count_event'], vert=False)
plt.xlabel('Количество событий на пользователя')
plt.title('Количество событий на пользователя')
plt.show()

По графику видим, что в данных есть выбросы, это пользователи на которых зафиксировано более 300 событий за неделю. Это могут быть как активные пользователи, так и боты. Посчитаем количество таких пользователей.

In [ ]:
# посмотрим распределение
percentiles = [ 0.9, 0.95, 0.99] 
column_percentiles = user_event_table['count_event'].quantile(percentiles)

print(column_percentiles)

In [ ]:
# смотрим сколько событий на пользователя отклоняются от  99 перцентиля
threshold = user_event_table['count_event'].quantile(0.99)
user_break = user_event_table[user_event_table['count_event'] > threshold].index.tolist()

print('Количество записей с выбросами в данных:',len(user_break))

In [ ]:
# cмотрим сколько записей приходится на аккаунты в которых есть выбросы
user_break_data = mobile_data.query('user_id in @user_break')['event_name'].count()
rate_break_data = len(user_break)/mobile_data['user_id'].nunique() * 100

print(f'Общее количество записей - {user_break_data}')
print(f'Процент выбросов от всех аккаунтов - {round(rate_break_data, 2)} %')


Удаление выбросов в этом случае приведет к потере 9626 строк. Данные оставляем.  


В результате предобработки данные очищены от дубликатов, добавлены новые столбцы, проведен анализ аномалий, изменен тип данный.

## 3. Исследователький анализ ##

###  Анализ активности пользователей ##


#### Количество событий в зависимости от времени ####

In [ ]:
# проверим, на все ли дни исследуемого месяца приходятся события. 
plt.figure(figsize=(12, 6))
ax = mobile_data['event_datetime'].hist(bins=500, alpha=0.9, color='#3A3238')

plt.title('Количество событий в зависимости от времени')
plt.ylabel("Дата")
plt.ylabel("Число событий")
plt.xticks(rotation=90)

plt.show()

Нормальное распределение в каждом из дней. Видим пик и спуски, в некоторые дни распределение смещено влево или вправо. Прослеживается связь между днями недели и активностью. 
Посмотрим на активность пользователей по дням недели

#### Активность пользователей по дням недели ####

In [ ]:
grouped_data = mobile_data.groupby('day_of_week').count()['event_name']

fig, ax = plt.subplots(figsize=(10, 4))
ax.bar(grouped_data.index, grouped_data.values, color='#9C9281')
ax.set_xlabel('День недели')
ax.set_ylabel('Количество событий')
ax.set_title('График количества событий по дням недели')

trend_line = np.polyfit(np.arange(len(grouped_data)), grouped_data.values, 1)
trend_values = np.polyval(trend_line, np.arange(len(grouped_data)))

ax.plot(grouped_data.index, trend_values, color='red', linestyle='--', label='Линия тренда')
ax.legend()
days_of_week = ['', 'Mon', 'Tue', 'Wed', 'Thu', 'Fri', 'Sat', 'Sun']
ax.set_xticklabels(days_of_week);

plt.show();


Линия тренда указывает на то, что активность пользователей снижается ближе к выходным, в воскресенье снова начинается рост активности.

####  Число уникальных пользователей в день - DAU ####


In [ ]:

dau_mobile_data = mobile_data.groupby('event_date')['user_id'].nunique()

trend_line = np.polyfit(np.arange(len(dau_mobile_data)), dau_mobile_data.values, 1)
trend_values = np.polyval(trend_line, np.arange(len(dau_mobile_data)))

fig, ax = plt.subplots(figsize=(15, 6))
ax.bar(dau_mobile_data.index, dau_mobile_data.values, color='#6B6570')
ax.plot(dau_mobile_data.index, trend_values, color='red', linestyle='--', label='Линия тренда')
ax.set_xlabel('Дата')
ax.set_ylabel('Число пользователей')
ax.set_title('График числа уникальных пользователей в день')
plt.xticks(rotation=90)
ax.legend()
plt.show()



Линия тренда указывает на рост числа уникальных пользователей на протяжении всего исследуемого периода. Видимо каналы привлечения клиентов работают, рекомендация - поддерживать рекламу в каналах привлечения.

####  Число уникальный пользователей в неделю - WAU ####

In [ ]:


mobile_data['date'] = mobile_data['event_time'].dt.date
mobile_data['session_on_week'] = mobile_data['date'].apply(lambda x: x.isocalendar()[1])

wau = mobile_data.groupby('session_on_week').agg({'user_id': 'nunique'}).reset_index()

fig, ax = plt.subplots(figsize=(10, 4))
sns.barplot(data=wau, x='session_on_week', y='user_id', color='#6B6D76')


plt.xlabel('Неделя')
plt.ylabel('Количество уникальных пользователей')
plt.title('Количество уникальных пользователей приложения в неделю')

plt.show()

Наблюдаем рост числа уникальных пользователей с первой по третью неделю, далее на четвертой неделе наблюдается снижение уникальных пользователей. 
Рекомендация - Проверить, что изменилось в приложении на последней недели исследования. 

####  Коэффициент удержания пользователей ####

In [ ]:

# дата начала первого сеанса для каждого пользователя
mobile_data['start_date'] = mobile_data.groupby('user_id')['event_time'].transform('min')
df = mobile_data[['user_id','event_datetime','start_date']]
df['dt'] = pd.to_datetime(df['start_date']).dt.date #дата первого посещения

# Рассчитатем лайфтайм
df['lifetime'] = (df['event_datetime'] - df['start_date']).dt.days

df.head()

In [ ]:
# задаём момент и горизонт анализа данных
moment = datetime(2019, 11, 3).date()
horizon_days = 14

# рассчитаем максимально возможную дату привлечения пользователя
maximum_date = moment - timedelta(days = horizon_days - 1)

# исключаем пользователей, пришедших после максимальной даты привлечения
df = df.query('dt <= @maximum_date')

In [ ]:
# создаем сводную для расчета retention
retention = df.pivot_table(index='dt', columns='lifetime', values='user_id', aggfunc='nunique')
# вычисляем размеры когорт
cohortes = (df.groupby('dt').agg({'user_id': 'nunique'}).rename(columns={'user_id': 'cohort_size'}))
# объединяем размеры когорт и таблицу удержания
retention = cohortes.merge(retention, on='dt', how='left').fillna(0)
# рассчитываем % удержания по когортам: 
retention = retention.div(retention['cohort_size'], axis=0)
# исключаем все лайфтаймы, превышающие горизонт анализа
retention = retention[['cohort_size'] + list(range(horizon_days))]
# восстанавливаем столбец с размерами когорт
retention['cohort_size'] = cohortes

In [ ]:
# построим хитмэп
plt.figure(figsize = (15, 6)) 
sns.heatmap(retention.drop(columns = ['cohort_size',0]), # удаляем размер когорт и 0 лайфтайм, потому что там 100%
            annot = True,  
            fmt = '.2%'  
           )
plt.title('Тепловая карта удержания')  
plt.show()

Судя по тепловой карте % удержания в разных когортах разный, неравномерный. Построим линейный график удержания. 

In [ ]:
# Построим кривые удержания:
report = retention.drop(columns = ['cohort_size', 0])

report.T.plot(
    grid= True,  
    xticks=list(report.columns.values),  # отметки на оси X — названия колонок
    figsize=(15, 5),  # размер графика
)
plt.xlabel('Лайфтайм')  # название оси X
plt.title('Кривые удержания по дням привлечения')  # название графика
plt.show()

Общая тенденция к снижению прослеживается, к 14 дню лайфтайма графики находятся в диапазоне от 0 до 7%. Посмотрим теперь удержание в динамике:

In [ ]:
report_dinamic = retention.drop(columns=['cohort_size', 0])
report_dinamic.plot(grid=True, figsize=(15, 5))
plt.xlabel('Дата привлечения')
plt.title('Динамика удержания пользователей')
plt.show()

Очень шумно, отфильтруем данные по 2 (потому что первый день = 100%), 7 и 14 дню лайфтайма: 

In [ ]:
report_dinamic = retention[[1,6,13]]
report_dinamic.plot(grid=True, figsize=(15, 5))
plt.xlabel('Дата привлечения')
plt.title('Динамика удержания пользователей')
plt.show()

После 17 октября наблюдаем снижение уровня удержания в когортах: графики 2, 7 и 14 дня демонтрируют снижение.

In [ ]:
# Построим усредненную кривую удержания, начиная со 2-го дня лайфтайма:
report_mean = retention.drop(columns = ['cohort_size',0])
report_mean = report_mean.mean()
report_mean.T.plot(
    grid= True,  
    xticks=list(report.columns.values),  # отметки на оси X — названия колонок
    figsize=(15, 5),  # размер графика
)
plt.xlabel('Лайфтайм')  # название оси X
plt.title('Усредненная кривая удержания')  # название графика
plt.show()

В среднем удержание не равномерно, резко падает со 100% до 12% на второй день, далее наблюдаем плато с 4 по 7 день, и стабилизацию удержания на уровне 2-4% с 9-го дня. 

 ##  Определение сессий ##

In [ ]:
# cортируем таблицу
mobile_data = mobile_data.sort_values(['user_id','event_time'])

In [ ]:
# устанавливаем Timedelta окончания сеанса использования приложения на уровне 30 мин бездействия. 
mobile_data['event_time'] = pd.to_datetime(mobile_data['event_time'])
g = (mobile_data.groupby('user_id')['event_time'].diff() > pd.Timedelta('30Min')).cumsum()

# записываем id сессий в столбец session_id
mobile_data['session_id'] = mobile_data.groupby(['user_id', g], sort=False).ngroup() + 1


###  Подготовка даных для определения популярных сценариев

In [ ]:
session_table = []
grouped_data = mobile_data.sort_values(by=['user_id', 'event_time']).groupby(['user_id', 'session_id'])

for (elem, el), group in grouped_data:
    event_in_session = group['event_name'].drop_duplicates().tolist()
    session_table.append([elem, el, event_in_session])
new_data = pd.DataFrame(session_table, columns=['user_id', 'session_id', 'event_in_session'])
session_grouped_data = new_data.groupby(new_data['event_in_session'].apply(tuple)).agg({
    'session_id': ['count', 'unique'],
    'user_id': ['count', 'unique']
})
session_grouped_data.columns = ['session_count', 'session_id', 'user_count', 'user_id'];
session_grouped_data = session_grouped_data.reset_index();
# оставляем только сессии c больше чем с одним действием и с целевым действием.
session_show_contacts = session_grouped_data[
    (session_grouped_data['event_in_session'].apply(lambda x: 'show_contacts' in x)) &
    (session_grouped_data['event_in_session'].apply(lambda x: len(x) != 1))]
 # выясняем id сессий в которых есть show_contacts
session_id_show_contacts = session_show_contacts['session_id'].tolist()
session_id_show_contacts = np.concatenate(session_id_show_contacts).tolist()

### Диаграмма Сэнки ###

Диаграмма подготовлена по примеру и с использованием кода из рекомендованного источника

In [ ]:
mobile_data = mobile_data.sort_values(by=['session_id', 'event_time'])

In [ ]:

table = mobile_data[['user_id', 'event_datetime', 'event_name', 'session_id']]
# далее удаляем дубликаты
table = table.drop_duplicates(subset=['session_id', 'event_name'])
# далее отбираем сессии с показом контактов
table = table[table['session_id'].isin(session_id_show_contacts)]


In [ ]:
def add_features(df):
    
    """Функция генерации новых столбцов для исходной таблицы

    """
    
    # сортируем по id и времени
    sorted_df = table.sort_values(by=['session_id', 'event_datetime']).copy()
    # добавляем шаги событий
    sorted_df['step'] = sorted_df.groupby('session_id').cumcount() + 1
    
    # добавляем узлы-источники и целевые узлы
    # узлы-источники - это сами события
    sorted_df['source'] = sorted_df['event_name']
    # добавляем целевые узлы
    sorted_df['target'] = sorted_df.groupby('session_id')['source'].shift(-1)
    
    # возврат таблицы без имени событий
    return sorted_df#.drop(['event_name'], axis=1)

table = add_features(table)

In [ ]:
def get_source_index(df):
    
    """Функция генерации индексов source

    """   
    res_dict = {}
    
    count = 0
    # получаем индексы источников
    for no, step in enumerate(df['step'].unique().tolist()):
        # получаем уникальные наименования для шага
        res_dict[no+1] = {}
        res_dict[no+1]['sources'] = df[df['step'] == step]['source'].unique().tolist()
        res_dict[no+1]['sources_index'] = []
        for i in range(len(res_dict[no+1]['sources'])):
            res_dict[no+1]['sources_index'].append(count)
            count += 1
            
    # соединим списки
    for key in res_dict:
        res_dict[key]['sources_dict'] = {}
        for name, no in zip(res_dict[key]['sources'], res_dict[key]['sources_index']):
            res_dict[key]['sources_dict'][name] = no
    return res_dict

In [ ]:
test_df = table.groupby('session_id')['event_name'].nunique()

# Визуализируем распределение уникальных событий в разрезе сессий
plt.hist(test_df, bins='auto')
plt.xlabel('События на сессию')
plt.ylabel('Число событий')
plt.title('Число уникальных событий в разрезе сессий')
plt.show()

Видим, что максимальное число событий на сессию - 6. Но по распределению числа событий на графике популярным можно назвать сценарии до 4 шагов. Далее по диаграмме увидим популярные сессии и максимальное число шагов в них.

In [ ]:
df_comp = table[table['step'] <= 4].copy().reset_index(drop=True)

In [ ]:
source_indexes = get_source_index(df_comp)

In [ ]:
def show_example(step, source_indexes=source_indexes):
    
    """Функция для вывода данных для конкретного шага будущей диаграммы

    """
    
    print(f'Пример подготовленных данных для шага {step}\n')
    for key in source_indexes[step]:
        print(f'{key}\n', source_indexes[step][key], '\n')

In [ ]:

def colors_for_sources(mode):
    
    """Генерация цветов rgba

    """
    # словарь, в который сложим цвета в соответствии с индексом
    colors_dict = {}
    
    if mode == 'random':
        for label in df_comp['source'].unique():
            r, g, b = np.random.randint(255, size=3)            
            colors_dict[label] = f'rgba({r}, {g}, {b}, 1)'
            
    elif mode == 'custom':
        colors = requests\
        .get('https://raw.githubusercontent.com/rusantsovsv/senkey_tutorial/main/json/colors_senkey.json').json()
        for no, label in enumerate(df_comp['source'].unique()):
            colors_dict[label] = colors['custom_colors'][no]
            
    return colors_dict

In [ ]:
colors_dict = colors_for_sources(mode='custom')

In [ ]:
# пересчитаем количестов юзеров в процентах от входа
def percent_users(sources, targets, values):
    
    """
    Расчет уникальных id в процентах (для вывода в hover text каждого узла)

    """
    
    # объединим источники и метки и найдем пары
    zip_lists = list(zip(sources, targets, values))
    
    new_list = []
    
    # подготовим список словарь с общим объемом трафика в узлах
    unique_dict = {}
    
    # проходим по каждому узлу
    for source, target, value in zip_lists:
        if source not in unique_dict:
            # находим все источники и считаем общий трафик
            unique_dict[source] = 0
            for sr, tg, vl in zip_lists:
                if sr == source:
                    unique_dict[source] += vl
                    
    # считаем проценты
    for source, target, value in zip_lists:
        new_list.append(round(100 * value / unique_dict[source], 1))
    
    return new_list

In [ ]:
def lists_for_plot(source_indexes=source_indexes, colors=colors_dict, frac=10):

    """
    Создаем необходимые для отрисовки диаграммы переменные списков 
    
    """
    
    sources = []
    targets = []
    values = []
    labels = []
    link_color = []
    link_text = []

    # проходим по каждому шагу
    for step in tqdm(sorted(df_comp['step'].unique()), desc='Шаг'):
        if step + 1 not in source_indexes:
            continue

        # получаем индекс источника
        temp_dict_source = source_indexes[step]['sources_dict']

        # получаем индексы цели
        temp_dict_target = source_indexes[step+1]['sources_dict']

        # проходим по каждой возможной паре, считаем количество таких пар
        for source, index_source in tqdm(temp_dict_source.items()):
            for target, index_target in temp_dict_target.items():
                # делаем срез данных и считаем количество id            
                temp_df = df_comp[(df_comp['step'] == step)&(df_comp['source'] == source)\
                                  &(df_comp['target'] == target)]
                value = len(temp_df)
                # проверяем минимальный объем потока и добавляем нужные данные
                if value > frac:
                    sources.append(index_source)
                    targets.append(index_target)
                    values.append(value)
                    # делаем поток прозрачным для лучшего отображения
                    link_color.append(colors[source].replace(', 1)', ', 0.2)'))
                    
    labels = []
    colors_labels = []
    for key in source_indexes:
        for name in source_indexes[key]['sources']:
            labels.append(name)
            colors_labels.append(colors[name])
            
    # посчитаем проценты всех потоков
    perc_values = percent_users(sources, targets, values)
    
    # добавим значения процентов для howertext
    link_text = []
    for perc in perc_values:
        link_text.append(f"{perc}%")
    
    # возвратим словарь с вложенными списками
    return {'sources': sources, 
            'targets': targets, 
            'values': values, 
            'labels': labels, 
            'colors_labels': colors_labels, 
            'link_color': link_color, 
            'link_text': link_text}

In [ ]:
data_for_plot = lists_for_plot();

In [ ]:
def plot_senkey_diagram(data_dict=data_for_plot):    
    
    """
    Функция для генерации объекта диаграммы Сенкей 
    
    """
    
    fig = go.Figure(data=[go.Sankey(
        domain = dict(
          x =  [0,1],
          y =  [0,1]
        ),
        orientation = "h",
        valueformat = ".0f",
        node = dict(
          pad = 50,
          thickness = 15,
          line = dict(color = "black", width = 0.1),
          label = data_dict['labels'],
          color = data_dict['colors_labels']
        ),
        link = dict(
          source = data_dict['sources'], # indices correspond to labels, eg A1, A2, A1, B1, ...
          target = data_dict['targets'],
          value = data_dict['values'],
          label = data_dict['link_text'],
          color = data_dict['link_color']
      ))])
    fig.update_layout(title_text="Sankey Diagram", font_size=10, width=900, height=520)
    
    # возвращаем объект диаграммы
    return fig


In [ ]:
senkey_diagram = plot_senkey_diagram()

In [ ]:
senkey_diagram.show()

### Определение сессий с помощью таблицы

In [ ]:
session_show_contacts['count_session'] = session_show_contacts\
                                .groupby('event_in_session')['session_id'].transform('count')
session_show_contacts = session_show_contacts.sort_values('session_count', ascending=False)
session_show_contacts.head(6)

session_show_contacts[['event_in_session', 'session_count']].head(6)


Можно выделить 6 самых популярных сценариев:
1. tips_show - show_contacts  - 342 сессии  
2. show_contacts - contacts_call  - 120 сессий  - это сценарий где 100 пользователей сразу приходят в целевое действие. Скорее всего они заходят в приложение сразу по ссылке на обьявление.
3. map - tips_show - show_contacts - 94 сессии  
4. photos_show - show_contacts - 85 сессий  
5. show_contacts - tips_show - 74 сессии  
6. search - show_contacts - contacts_call - 54 сессии  
Визуализируем некоторые из них

### Воронка по сценарию "tips_show -> show_contacts''

In [ ]:
# считаем общее число уникальных пользователей на этапе tips_shows
step_1 = mobile_data.query('event_name=="tips_show"')['user_id'].nunique()
# список уникальных пользователей на этапе tips_show 
list_users = mobile_data.query('event_name=="tips_show"')['user_id'].unique().tolist()

# считаем число уникальных пользователей на этапе show_contacts, из тех, что были на этапе tips_show
step_2 = mobile_data.query('user_id in @list_users and event_name=="show_contacts"')['user_id'].nunique()


In [ ]:
# строим воронку 
funnel_labels = ['tips_show', 'show_contacts']
funnel_counts = [step_1, step_2]

fig = go.Figure(go.Funnel(
    y = funnel_labels,
    x = funnel_counts,
    textinfo = "value+percent initial"))

fig.update_layout(
    title="Воронка по сценарию \"tips_show -> show_contacts\"",
    xaxis_title="Количество пользователей",
    yaxis_title="Этапы воронки",
    width = 800
)

fig.show()

В первом по популярности сценарии пользователи кликают на показанное обьявление и только 18% пользователей доходят до целевого действия

### Воронка по сценарию "map -> tips_show -> show_contacts" ###

In [ ]:
# считаем общее число уникальных пользователей на этапе map  
step_1 = mobile_data.query('event_name=="map"')['user_id'].nunique()
# список уникальных пользователей на этапе tips_show
list_users_1 = mobile_data.query('event_name=="map"')['user_id'].unique().tolist()

# считаем число уникальных пользователей на этапе tips_show, из тех, что были на этапе map
step_2 = mobile_data.query('user_id in @list_users_1 and event_name=="tips_show"')['user_id'].nunique()
list_users_2 = mobile_data.query('user_id in @list_users_1 and event_name=="tips_show"')['user_id']\
                          .unique()\
                          .tolist()

# считаем число уникальных пользователей на этапе show_contacts, из тех, что были на этапе tips_show
step_3 = mobile_data.query('user_id in @list_users_2 and event_name=="show_contacts"')['user_id'].nunique()


In [ ]:
# строим воронку 
funnel_labels = ['map', 'tips_show', 'show_contacts']
funnel_counts = [step_1, step_2, step_3]

fig = go.Figure(go.Funnel(
    y = funnel_labels,
    x = funnel_counts,
    textinfo = "value+percent initial"))

fig.update_layout(
    title="Воронка по сценарию \"map -> tips_show -> show_contacts\"",
    xaxis_title="Количество пользователей",
    yaxis_title="Этапы воронки",
    width = 800
)

fig.show()

19% пользователей доходят до целевого действия в данном сценарии.Большая часть людей теряется в этом сценарии на этапе tips_show - show_contact. Возможно следует пересмотреть алгоритм рекомендации обьявлений, это может увеличить конверсию на данном этапе

### c) Воронка по сценарию "photos_show -> show_contacts"

In [ ]:
# считаем общее число уникальных пользователей на этапе photos_show
step_1 = mobile_data.query('event_name=="photos_show"')['user_id'].nunique()
# список уникальных пользователей на этапе tips_show
list_users = mobile_data.query('event_name=="photos_show"')['user_id'].unique().tolist()

# считаем число уникальных пользователей на этапе show_contacts, из тех, что были на этапе photos_show
step_2 = mobile_data.query('user_id in @list_users and event_name=="show_contacts"')['user_id'].nunique()

In [ ]:
# строим воронку 
funnel_labels = ['photos_show', 'show_contacts']
funnel_counts = [step_1, step_2]

fig = go.Figure(go.Funnel(
    y = funnel_labels,
    x = funnel_counts,
    textinfo = "value+percent initial"))

fig.update_layout(
    title="Воронка по сценарию \"photos_show -> show_contacts\"",
    xaxis_title="Количество пользователей",
    yaxis_title="Этапы воронки",
    width = 800
)

fig.show()


31% пользователей после просмотра фото доходит до целевого дейтсвий.

### d) Воронка по сценарию "search -> show_contacts -> contacts_call" ###

In [ ]:
# считаем общее число уникальных пользователей на этапе search  
step_1 = mobile_data.query('event_name=="search"')['user_id'].nunique()
# список уникальных пользователей на этапе tips_show
list_users_1 = mobile_data.query('event_name=="search"')['user_id'].unique().tolist()

# считаем число уникальных пользователей на этапе show_contacts, из тех, что были на этапе search
step_2 = mobile_data.query('user_id in @list_users_1 and event_name=="show_contacts"')['user_id'].nunique()
list_users_2 = mobile_data.query('user_id in @list_users_1 and event_name=="show_contacts"')['user_id']\
                          .unique()\
                          .tolist()

# считаем число уникальных пользователей на этапе contacts_call, из тех, что были на этапе show_contacts
step_3 = mobile_data.query('user_id in @list_users_2 and event_name=="contacts_call"')['user_id'].nunique()


In [ ]:
# строим воронку 
funnel_labels = ['map', 'tips_show', 'show_contacts']
funnel_counts = [step_1, step_2, step_3]

fig = go.Figure(go.Funnel(
    y = funnel_labels,
    x = funnel_counts,
    textinfo = "value+percent initial"))

fig.update_layout(
    title="Воронка по сценарию \"search -> show_contacts -> contacts_call\"",
    xaxis_title="Количество пользователей",
    yaxis_title="Этапы воронки",
    width = 800
)

fig.show()

В данном сценарии 23% пользователе посте поиска переходят на просмотр контактов - целевое действие. Только 7% пользователей звонят по обьявлению. 

Самый популярный сценарий - tips_show -> show_contacts. Люди видят обьявление и переходят в контакты.
Больший всего процент пользователей(31%) доходит до целевого действий по сценарию - photos_show -> show_contacts
Пользователей переходят на просмотр контактов после показа фото. Рекомендация - стоит ввести требования к фотографии, так как качетсво фото влияет на достижение пользователем целевого действия. 

Боьше всего людей теряется теряется на этапе tips_show в воронке map -> tips_show -> show_contacts. Возможно следует пересмотреть алгоритм рекомендации обьявлений, это может увеличить конверсию на данном этапе

## Оценить, какие действия чаще совершают те пользователи, которые просматривают контакты. ##

Рассчитать относительную частоту событий в разрезе двух групп пользователей:
○ группа пользователей, которые смотрели контакты contacts_show
○ группа пользователей, которые не смотрели контакты contacts_show

In [ ]:
# пользователи, посмотревшие контакты
list_show_contacts = mobile_data.query('event_name =="show_contacts"')['user_id'].unique().tolist()
show_contacts = mobile_data.query('user_id in @list_show_contacts')

In [ ]:
# строим сводную таблицу по тем, кто посмотрел контакты
# группируем по событиям, считаем количество сессий по событиям и количество уникальных сессий
pivot_show_contacts = show_contacts.groupby('event_name')\
                                   .agg({'session_id':['count','nunique']})\
                                   .reset_index()
pivot_show_contacts.columns = ['event_name','session_count', 'session_unique']

# считаем среднее количество событий за сессию
pivot_show_contacts['count_per_session'] = round\
                                (pivot_show_contacts['session_count']\
                                 /pivot_show_contacts['session_unique'],2)
# считаем относительное количество событий по отношению к общему числу событий
pivot_show_contacts['ratio_show'] = round(pivot_show_contacts['session_count']\
                                          /pivot_show_contacts['session_count']\
                                          .sum(),2)
pivot_show_contacts.sort_values('ratio_show', ascending = False)

In [ ]:
# пользователи, не посмотревшие контакты
list_non_show_contacts = mobile_data.query('user_id not in @list_show_contacts')['user_id'].unique().tolist()
non_show_contacts = mobile_data.query('user_id in @list_non_show_contacts')

In [ ]:
# расчет таблицы аналогично предыдущей
pivot_non_show_contacts = non_show_contacts.groupby('event_name')\
                                            .agg({'session_id':['count','nunique']})\
                                            .reset_index()
pivot_non_show_contacts.columns = ['event_name','session_count', 'session_unique']
pivot_non_show_contacts['count_per_session'] = round(pivot_non_show_contacts['session_count']\
                                                    / pivot_non_show_contacts['session_unique'],2)
pivot_non_show_contacts['ratio_non_show'] = round(pivot_non_show_contacts['session_count']\
                                                  /pivot_non_show_contacts['session_count'].sum(),2)
pivot_non_show_contacts.sort_values('ratio_non_show', ascending = False)

In [ ]:
# строим барплот для сравнения частоты событий по группам пользователей 

compare = pivot_show_contacts.merge(pivot_non_show_contacts, on = 'event_name', \
                                    how = 'left', suffixes=('_show','non_show'))\
                                    .sort_values('ratio_show')

import plotly.express as px
fig = px.bar(compare, y="event_name", x=['ratio_show','ratio_non_show'], barmode = 'group', 
            title = 'Относительная частота событий', 
            labels={'event_name':'Событие', 'value':'Относительная частота'},
            )
fig.update_layout(legend_title_text='Пользователи')
fig.show()


На первом месте у обоих групп - переход по обьявлению. Далее действия различаются.
  
1. Первая группа - те, кто просматривал контакты. 
Самое частое действие (кроме открытия обьявления и просмотра контактов) - просмотр фото, поиск и открытие карточки

2. Вторая группа - те, кто не просматривал контакты
Самое частое действие(кроме открытия обьявления) - просмотр фото,открытие карточки объявления и поиск.

Нет значимых различий в пользовательских сценариях групп тех юзеров, которые просматривают контакты и тех кто не просматривают. 


## Проверка гипотез

### Основная гипотеза ###

Одни пользователи совершают действия tips_show и tips_click, другие только tips_click.  Проверить гипотезу - Конверсия и просмотры контактов различается у этих групп.  

Создаем две группы пользователей. Первая - которая совершает дейтсвие и tips_show и tips_click;
Вторая группа - пользователи, которые совершают только tips_click  
       
- Нулевая гипотеза - Данные групп по конверсии и просмотру контактов не различаются
- Альтернативная гипотеза - Есть различия в конверсии у двух выделенных групп

In [ ]:
# отберем пользователей, которые совершили tips_show:
list_total_users = mobile_data.query('event_name=="tips_show"')['user_id'].unique().tolist()
# датасет с такими пользователями
total_users = mobile_data.query('user_id in @list_total_users')
print('Всего пользователей, совершивших tips_show:', len(list_total_users))

In [ ]:
# Получим датасет с теми пользователями, кто совершил оба действия:
# список пользователей, совершивших оба действия:
list_users_tips_show_click = total_users.query('event_name=="tips_click"')['user_id'].unique().tolist()
users_tips_show_click = total_users.query('user_id in @list_users_tips_show_click')
print('Всего пользователей, совершивших (tips_show + tips_click):', users_tips_show_click['user_id'].nunique())

In [ ]:
# Получим датасет с теми пользователями, кто совершил только  tips_click:

# формируем датасет, исключая список с теми пользователями, кто совершил оба действия:
users_tips_show = total_users.query('user_id not in @list_users_tips_show_click')
print('Всего пользователей, совершивших только tips_show:', users_tips_show['user_id'].nunique())

Получили два датасета, с неравным количеством пользователей. Что логично, далеко не все пользователи открывают все объявления, которые видят. 

In [ ]:
''' Считаем conversion_rates по группам. Затем проверяем гипотезу z_тестом'''

# количество уникальных пользователей на этапе show_contacts по группам
successes_tips_show_click = users_tips_show_click[users_tips_show_click['event_name']=="show_contacts"]\
                                                                                ['user_id'].nunique()
successes_tips_show = users_tips_show[users_tips_show['event_name']=="show_contacts"]['user_id']\
                                                                                        .nunique()

# всего пользователей в группах
totals_tips_show_click = users_tips_show_click['user_id'].nunique()
totals_tips_show = users_tips_show['user_id'].nunique()

# рассчитаем конверсии по группам: 
conversion_rate_tips_show_click = (successes_tips_show_click / totals_tips_show_click)*100
conversion_rate_tips_show = (successes_tips_show /totals_tips_show)*100

print("Conversion Rates:")
print('Конверсия в группе "tips_show + tips_click"', "{:.2f}%".format(conversion_rate_tips_show_click))
print('Конверсия в группе "tips_show"', "{:.2f}%".format(conversion_rate_tips_show))
    
from statsmodels.stats.proportion import proportions_ztest

# количество пользователей, совершивших целевое действие
successes = [successes_tips_show_click, successes_tips_show]
totals = [totals_tips_show_click, totals_tips_show]

stat, p_value = proportions_ztest(successes, totals)

alpha = 0.05
print('p_value:',p_value)
# Проверка гипотезы
if p_value < alpha:
    print("Отвергаем нулевую гипотезу. Конверсия в первой и второй группе различается.")
else:
    print("Не получилось опровергнуть нулевую гипотезу.\
    Конверсия в первой и второй группе не имеет существенных различий.")



Конверсия в группах 1 и 2  различается. Конверсия в группе, которая и смотрела обьявление и кликала на него почти в 2 раза выше групы, которая только видела обьявление.
    

### 5.2. Дополнительная гипотеза ###

Проверяем, влияет ли источник привлечения клиентов на совершение ими целевого действия - show_contacts. Для проверки будем использовать 2 источника yandex и google  
Нулевая гипотеза: разницы между конверсиями у выборок нет     
Альтернативная гипотеза: разница между конверсиями выборок присутствует

Выделяем три группы в зависимости от источника привлечения: yandex, goodle, other

In [ ]:
# обьединяем данные из таблиц mobile_sources и new_data
source_mapping = mobile_sources.set_index('user_id')['source']
mobile_data['source'] = mobile_data['user_id'].map(source_mapping)

In [ ]:
''' Считаем conversion_rates по группам. Затем проверяем гипотезу z_тестом'''
grouped_data_source = mobile_data.groupby('source')
conversion_rates = {}

for group, group_data in grouped_data_source:
    total_users = group_data['user_id'].nunique()
    target_action_users = group_data[group_data['event_name']=='show_contacts']['user_id'].nunique()

    conversion_rate = (target_action_users / total_users) * 100

    conversion_rates[group] = conversion_rate

print("Conversion Rates:")
for group, rate in conversion_rates.items():
    print("Group {}: {:.2f}%".format(group, rate))
    
successes_source = [
    mobile_data[(mobile_data['event_name'].apply(lambda actions: 'show_contacts' in actions))\
                & (mobile_data['source'] == 'yandex')]['user_id'].nunique(),
    mobile_data[(mobile_data['event_name'].apply(lambda actions: 'show_contacts' in actions))\
                & (mobile_data['source'] == 'google')]['user_id'].nunique()                
                    ]

totals_source = [
    mobile_data[mobile_data['source'] == 'yandex']['user_id'].nunique(),
    mobile_data[mobile_data['source'] == 'google']['user_id'].nunique()
                ]
stat, p_value = proportions_ztest(successes_source, totals_source)

alpha = 0.05
print('p_value:', p_value)

# Проверка гипотезы
if p_value < alpha:
    print("Отвергаем нулевую гипотезу. Конверсия у разных источников различается.")
else:
    print("Не получилось опровергнуть нулевую гипотезу. Нет различия в конверсиях у источников привлечения.")



После проведения проверки гипотезы мы приходим к выводу, что конверсия во всех источниках рекламы одинаковая. Следовательно, мы можем оптимизировать бюджет на рекламу, сделать акцент на более дешевом источнике.


## Выводы ##



В результате исследования датасетов приложения «Ненужные вещи» получили следущие выводы и рекомендации.Общий обзор пользовательских действий показал -   Число уникальных пользователей приложения растет на протяжении исследуемого периода. 
В разрезе по неделям фиксируем рост числа уникальных пользователей с первой по третью неделю, далее на четвертой неделе наблюдается снижение уникальных пользователей. В разрезе дней недели - активность пользователей снижается ближе к выходным и повышается в начале недели. Это можно учитывать при планировании рекламы в приложении. 

Анализ удержания. В среднем удержание не равномерно, резко падает со 100% до 12% на второй день, далее наблюдаем плато с 4 по 7 день, и стабилизацию удержания на уровне 2-4% с 9-го дня. 

**1. Ответ на вопросы заказчика** 

1.1. Проанализируйте связь целевого события —
просмотра контактов — и других действий
пользователей:  

По результатам анализа сессии можно выделить 6 самых популярных сценариев пользователей: 
- tips_show - show_contacts - 342 сессии   
- show_contacts - contacts_call - 120 сессий - это сценарий где 100 %   пользователей приходят в целевое действие. Скорее всего они заходят в приложение сразу по ссылке на объявление.   
- map - tips_show - show_contacts - 94 сессии   
- photos_show - show_contacts - 85 сессий   
- show_contacts - tips_show - 74 сессии   
- search - show_contacts - contacts_call - 54 сессии  

Построены воронки. Самый популярный сценарий - tips_show -> show_contacts. Люди видят обьявление и переходят в контакты.
Больший всего процент пользователей(31%) доходит до целевого действий по сценарию - photos_show -> show_contacts
Пользователей переходят на просмотр контактов после показа фото. Рекомендация - стоит ввести требования к фотографии, так как качетсво фото влияет на достижение пользователем целевого действия. 

Боьше всего людей теряется теряется на этапе tips_show в воронке map -> tips_show -> show_contacts. Возможно следует пересмотреть алгоритм рекомендации обьявлений, это может увеличить конверсию на данном этапе 
*Рекомендация - Возможно следует пересмотреть алгоритм рекомендации обьявлений в картах, это может увеличить конверсию на данном этапе.

1.2. Рассчитать относительную частоту событий в разрезе двух групп пользователей:
--группа 1. пользователей, которые смотрели контакты contacts_show
--группа 2. пользователей, которые не смотрели контакты contacts_show

Исследование показало, что частота событий в обоих группа не сильно различается, кроме факта просмотра контактов. 



**2. Проверка  гипотез** 
Гипотеза 1: Одни пользователи совершают действия tips_show и tips_click, другие только tips_click.  Проверить гипотезу - Конверсия и просмотры контактов различается у этих групп.  

Конверсия в группах 1 и 2 различается. Конверсия в группе, которая и смотрела обьявление и кликала на него почти в 2 раза выше групы, которая только видела обьявление. 

Гипотеза 2 (дополнительная): Проверяем, влияет ли источник привлечения клиентов на совершение ими целевого действия - show_contacts. Для проверки будем использовать 2 источника yаndeх и google
Гипотеза - есть связь между целевым действием и источником привлечения.
После проведения проверки гипотезы мы приходим к выводу, что конверсия во всех источниках рекламы одинаковая. Следовательно, рекомендуем оптимизировать бюджет на рекламу, сделать акцент на более дешевом источнике.
